In [8]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glove_routines import *
%load_ext autoreload
%autoreload 2
    
# load in data and labels    
#data = np.array(np.loadtxt('data/data.txt'))
pos_train = open('data/pos_train.txt').readlines()
neg_train = open('data/neg_train.txt').readlines()
embeddings = np.load('data/embeddings.npy')
vocab = open('data/vocab.txt')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import re

def findWholeWord(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

def construct_features():
    #Load the training tweets and the built GloVe word embeddings.
    
    #construct a feature representation of each training tweet 
    #(by averaging the word vectors over all words of the tweet).
    word_nbr_per_tweet_pos = np.zeros(np.shape(pos_train)[0])
    for j in range(0,np.shape(pos_train)[0]):
        tweet = pos_train[j]
        size = len(re.findall(r'\w+', tweet))
        word_nbr_per_tweet_pos[j] = size
        
    word_nbr_per_tweet_neg = np.zeros(np.shape(neg_train)[0])
    for j in range(0,np.shape(neg_train)[0]):
        tweet = neg_train[j]
        size = len(re.findall(r'\w+', tweet))
        word_nbr_per_tweet_neg[j] = size
    
    print("Counting words in tweet finished.")
    i=0
    pos_mask = np.zeros(np.shape(embeddings)[1]+1)
    pos_mask[0] +=1
    training_set_pos = np.zeros(((np.shape(pos_train)[0],np.shape(embeddings)[1]+1))) + pos_mask
    training_set_neg = np.zeros(((np.shape(neg_train)[0],np.shape(embeddings)[1]+1)))
    vocab = open('data/vocab_cut.txt')
    for word in vocab:
        if(i%1000==0):
            print("Still running.",i,"words done.")
        current_emb = embeddings[i]
        for j in range(0,np.shape(pos_train)[0]):
            if word in pos_train[j]:
                training_set_pos[j,1:np.shape(embeddings)[1]+1] += current_emb
        for j in range(0,np.shape(neg_train)[0]):
            if word in neg_train[j]:
                training_set_neg[j,1:np.shape(embeddings)[1]+1] += current_emb
        i+=1
    print("All words done.")
    for i in range(0,20):
        training_set_pos[:,i+1] = training_set_pos[:,i+1]/word_nbr_per_tweet_pos
        training_set_neg[:,i+1] = training_set_neg[:,i+1]/word_nbr_per_tweet_neg
    return training_set_pos,training_set_neg
    

tsp, tsn = construct_features()
np.save('data/trainingset_pos', tsp)
np.save('data/trainingset_neg', tsn)

Running. i is: 0
Running. i is: 1000
Running. i is: 2000
Running. i is: 3000
Running. i is: 4000
Running. i is: 5000
Running. i is: 6000
Running. i is: 7000
Running. i is: 8000


KeyboardInterrupt: 

In [22]:
import re
word_nbr_per_tweet_pos = np.zeros(np.shape(pos_train)[0])
for j in range(0,np.shape(pos_train)[0]):
    tweet = pos_train[j]
    size = len(re.findall(r'\w+', tweet))
    word_nbr_per_tweet_pos[j] = size

training_set_pos = np.zeros(((np.shape(pos_train)[0],np.shape(embeddings)[1]+1)))
for i in range(0,20):
    training_set_pos[:,i+1] = training_set_pos[:,i+1]/word_nbr_per_tweet_pos
print(np.shape(training_set_pos))
print(np.shape(training_set_pos[:,5]))

(100000, 21)
(100000,)


In [6]:
pos_train = open('data/pos_train.txt').readlines()
print(np.shape(pos_train))
for i in range(0,np.shape(pos_train)[0]):
    i+=1

(100000,)
